In [1]:
%load_ext py_d3

In [29]:
%%d3 4.4.4

<svg width=800 height=400></svg>

<script>
    requirejs.config({
        paths: {
            tree: "https://unpkg.com/threshold-tree@1.0.4/umd/threshold-tree"
        }
    });
    
    requirejs(['tree'], function(tree) {
        
        d3.csv("../data/complaint_types_simplified.csv", function(complaints) {

            // Build the threshold tree and convert it to a simple JSON format.
            let hr = new tree.ThresholdTree(complaints).as_hierarchy();
            
            console.log(hr);
            
            // Convert that to a hierarchy.
            let root = d3.stratify().id(d => d.name).parentId(d => d.parent)(hr);
            
            // Attach a summer to the hierarchy.
            root = root.sum(d => d.n);
            
            // console.log(root);
            
            // Create our treemap layout factory function.
            let treemap = d3.treemap()
                .tile(d3.treemapResquarify)
                .size([800, 400])
                .round(true)
                .paddingInner(1);
            
            // Apply our treemap function to our data.
            treemap(root);
            
            // console.log(root);
            console.log(root.descendants());
            
            // Select the SVG.
            const svg = d3.select("svg");
            
            // Lay out the cells.
            let cell = svg.selectAll("g")
                // .data(root.descendants().slice(1))  // include the basal Air Quality node.
                .data(root.leaves())
                .enter()
                .append("g")
                .attr("transform", function(d) { return "translate(" + d.x0 + "," + d.y0 + ")"; });
            
            // console.log(root);
            // console.log(cell);

            cell.append("rect")
                .attr("id", function(d) { return d.data.id; })
                .attr("width", function(d) { return d.x1 - d.x0; })
                .attr("height", function(d) { return d.y1 - d.y0; })
                .attr("fill", function(d) { return "transparent"; })
                .attr("stroke", function(d) { return "black"; })
                .attr("stroke-width", "1");
        });
        
    });
</script>

In [73]:
%%d3 4.4.4

<svg width=800 height=400></svg>

<script>
    requirejs.config({
        paths: {
            tree: "https://unpkg.com/threshold-tree@1.0.4/umd/threshold-tree"
        }
    });
    
    requirejs(['tree'], function(tree) {
        
        d3.csv("../data/complaint_types.csv", function(complaints) {

            // Build the threshold tree and convert it to a simple JSON format.
            let tr = new tree.ThresholdTree(complaints);
            
            console.log(tr);
            
            let hr = tr.as_hierarchy();
            
            // Convert that to a hierarchy.
            let root = d3.stratify().id(d => d.name).parentId(d => d.parent)(hr);
            
            // Attach a summer to the hierarchy.
            root = root.sum(d => d.n);
            
            // Create our treemap layout factory function.
            let treemap = d3.treemap()
                .tile(d3.treemapResquarify)
                .size([800, 400])
                .round(true)
                .paddingInner(1);
            
            // Apply our treemap function to our data.
            treemap(root);
            
            // console.log(root);
            // root = root.children[0]

            // Select the SVG.
            const svg = d3.select("svg");
            
            // Lay out the cells.
            let cell = svg.selectAll("g")
                .data(root.leaves())
                .enter()
                .append("g")
                .attr("transform", function(d) { return "translate(" + d.x0 + "," + d.y0 + ")"; });
            
            // console.log(root);
            // console.log(cell);

            cell.append("rect")
                .attr("id", function(d) { return d.data.id; })
                .attr("width", function(d) { return d.x1 - d.x0; })
                .attr("height", function(d) { return d.y1 - d.y0; })
                .attr("fill", function(d) { return "transparent"; })
                .attr("stroke", function(d) { return "black"; })
                .attr("stroke-width", "1");
        });
        
    });
</script>

In [82]:
%%d3 4.4.4

<svg width=800 height=400></svg>

<script>
    requirejs.config({
        paths: {
            tree: "https://unpkg.com/threshold-tree@1.0.4/umd/threshold-tree"
        }
    });
    
    requirejs(['tree'], function(tree) {
        
        d3.csv("../data/complaint_types.csv", function(complaints) {

            // Build the threshold tree and convert it to a simple JSON format.
            let tr = new tree.ThresholdTree(complaints)
            
            tr.root = tr.root.children[0];  // sub-example
            // tr.root = tr.root.children[0].children[6];  // sub-sub-example
            tr.root.parent = null;
            
            // We have to set the n of the root node to null, and remove the sub-sub-nodes.
            // Otherwise these gets included in the count, making the figure smaller than it should be.
            tr.root.n = null;
            tr.root.children.forEach(function(node) { node.children = []; });
            
            console.log(tr);
            
            let hr = tr.as_hierarchy();
            
            // Convert that to a hierarchy.
            let root = d3.stratify().id(d => d.name).parentId(d => d.parent)(hr);
            
            // Attach a summer to the hierarchy.
            root = root.sum(d => d.n);
            
            // Create our treemap layout factory function.
            let treemap = d3.treemap()
                .tile(d3.treemapResquarify)
                .size([800, 400])
                .round(true)
                .paddingInner(1);
            
            // Apply our treemap function to our data.
            treemap(root);

            // Select the SVG.
            const svg = d3.select("svg");
            
            // Lay out the cells.
            let cells = svg.selectAll("g")
                .data(root.leaves())
                .enter()
                .append("g")
                .attr("transform", function(d) { return "translate(" + d.x0 + "," + d.y0 + ")"; });
            
            cells.append("rect")
                .attr("id", function(d) { return d.data.id; })
                .attr("width", function(d) { return d.x1 - d.x0; })
                .attr("height", function(d) { return d.y1 - d.y0; })
                .attr("fill", function(d) { return "transparent"; })
                .attr("stroke", function(d) { return "black"; })
                .attr("stroke-width", "1");
            
            console.log(cells);
            
        });
        
    });
</script>

In [126]:
%%d3 4.4.4

<svg width=800 height=400></svg>

<script>
    requirejs.config({
        paths: {
            tree: "https://unpkg.com/threshold-tree@1.0.4/umd/threshold-tree"
        }
    });
    
    requirejs(['tree'], function(tree) {
        
        d3.csv("../data/complaint_types.csv", function(complaints) {

            // Build the threshold tree and convert it to a simple JSON format.
            let tr = new tree.ThresholdTree(complaints);
            
            // tr.root = tr.root.children[0];  // sub-example
            // tr.root = tr.root.children[0].children[6];  // sub-sub-example
            tr.root.parent = null;
            
            // We have to set the n of the root node to null, and remove the sub-sub-nodes.
            // Otherwise these gets included in the count, making the figure smaller than it should be.
            tr.root.n = null;
            
            let hr = tr.as_hierarchy();
            
            // Convert that to a hierarchy.
            let root = d3.stratify().id(d => d.name).parentId(d => d.parent)(hr);
            
            // Attach a summer to the hierarchy.
            // This summer needs to account for two factors:
            // 1. It should not include the n of the root node, as this will double to display space.
            // 2. It should not include sub-sub-children, as D3 recurses naively and doesn't care about
            //    how deep our objects are.
            // These two things were discovered experimentally.
            // Unfortunately, Set.has doesn't work on Objects. See:
            // http://stackoverflow.com/questions/29759480/how-to-customize-object-equality-for-javascript-set
            // So we have to implement a method for checking if an Object is a subchild ourselves. This is
            // the below.
            //
            // Note: a simpler solution is to simply replace the list of children with an empty list back when
            // we first made the list. e.g.: tr.root.children.forEach(function(node) { node.children = []; });,
            // but we need to keep that data around for transitions, not just delete it, so we don't do that.
            function object_is_direct_child(root, obj) {
                return (root.children.filter(child => (child.id === obj.name)).length > 0);
            }
            
            // Now the sum function itself.
            root = root.sum(function(node) {
                return ((object_is_direct_child(root, node)) ? node.n : 0);
            });
            
            // Create our treemap layout factory function.
            let treemap = d3.treemap()
                .tile(d3.treemapResquarify)
                .size([800, 400])
                .round(true)
                .paddingInner(1);
            
            // Apply our treemap function to our data.
            treemap(root);

            // Select the SVG.
            const svg = d3.select("svg");
            
            // To lay out the cells, we would typically make use of root.leaves(), which fetches all
            // of the leaf nodes of the hierarchy and returns them. This is used for the data bind.
            // However, in our case we don't want the leaves, because these are the items at the bottom
            // of our hierarchy. We use root.children directly instead. Works fine.
            
            // Lay out the cells.
            let cell = svg.selectAll("g")
                .data(root.children)
                .enter()
                .append("g")
                .attr("transform", function(d) { return "translate(" + d.x0 + "," + d.y0 + ")"; });
            
            // console.log(root);
            // console.log(cell);

            cell.append("rect")
                .attr("id", d => d.data.id)
                .attr("width", d => d.x1 - d.x0)
                .attr("height", d => d.y1 - d.y0)
                .attr("fill", "steelblue")
                .attr("stroke", "white")
                .attr("stroke-width", "1");
        });
        
    });
</script>

In [176]:
%%d3 4.4.4

<svg width=800 height=400></svg>

<script>
    requirejs.config({
        paths: {
            tree: "https://unpkg.com/threshold-tree@1.0.4/umd/threshold-tree"
        }
    });
    
    requirejs(['tree'], function(tree) {
        
        d3.csv("../data/complaint_types.csv", function(complaints) {

            // Build the threshold tree and convert it to a simple JSON format.
            let tr = new tree.ThresholdTree(complaints);
            
            // tr.root = tr.root.children[0];  // sub-example
            // tr.root = tr.root.children[0].children[6];  // sub-sub-example
            tr.root.parent = null;
            
            // We have to set the n of the root node to null, and remove the sub-sub-nodes.
            // Otherwise these gets included in the count, making the figure smaller than it should be.
            tr.root.n = null;
            
            let hr = tr.as_hierarchy();
            
            // Convert that to a hierarchy.
            let root = d3.stratify().id(d => d.name).parentId(d => d.parent)(hr);
            
            // Attach a summer to the hierarchy.
            // This summer needs to account for two factors:
            // 1. It should not include the n of the root node, as this will double to display space.
            // 2. It should not include sub-sub-children, as D3 recurses naively and doesn't care about
            //    how deep our objects are.
            // These two things were discovered experimentally.
            // Unfortunately, Set.has doesn't work on Objects. See:
            // http://stackoverflow.com/questions/29759480/how-to-customize-object-equality-for-javascript-set
            // So we have to implement a method for checking if an Object is a subchild ourselves. This is
            // the below.
            //
            // Note: a simpler solution is to simply replace the list of children with an empty list back when
            // we first made the list. e.g.: tr.root.children.forEach(function(node) { node.children = []; });,
            // but we need to keep that data around for transitions, not just delete it, so we don't do that.
            function object_is_direct_child(root, obj) {
                return (root.children.filter(child => (child.id === obj.name)).length > 0);
            }
            
            // Now the sum function itself.
            root = root.sum(function(node) {
                return ((object_is_direct_child(root, node)) ? node.n : 0);
            });
            
            // Create our treemap layout factory function.
            let treemap = d3.treemap()
                .tile(d3.treemapResquarify)
                .size([800, 400])
                .round(true)
                .paddingInner(1);
            
            // Apply our treemap function to our data.
            treemap(root);

            // Select the SVG.
            const svg = d3.select("svg");
            
            // To lay out the cells, we would typically make use of root.leaves(), which fetches all
            // of the leaf nodes of the hierarchy and returns them. This is used for the data bind.
            // However, in our case we don't want the leaves, because these are the items at the bottom
            // of our hierarchy. We use root.children directly instead. Works fine.
            
            // Lay out the cells (element transform on groups)
            let cells = svg.selectAll("g")
                .data(root.children)
                .enter()
                .append("g")
                .attr("transform", d => `translate(${d.x0}, ${d.y0})`);
            
            // Append rects.
            let rects = cells.append("rect")
                .attr("id", d => `${d.data.name}-rect`)
                .attr("width", d => d.x1 - d.x0)
                .attr("height", d => d.y1 - d.y0)
                .attr("fill", "steelblue")
                .attr("stroke", "white")
                .attr("stroke-width", "1");
            
            // Append text labels.
            let texts = cells.append("text")
                .attr("id", d => `${d.data.name}-text`)
                .attr("transform", d => `translate(${(d.x1 - d.x0) / 2}, ${(d.y1 - d.y0) / 2})`)
                .attr("text-anchor", "middle")
                .attr("alignment-baseline", "middle")
                .text(d => d.data.name)
            
            // Add mouseover events signaling interactivity. We want these to fire regardless of whether
            // we are hoving over the text or the rect behind it, so we add invisible rects (same as the)
            // visible ones) to the screen on top, which have hover effects defined on them.
            let hover_rects = cells.append("rect")
                .attr("id", d => `${d.data.name}-hover`)
                .attr("width", d => d.x1 - d.x0)
                .attr("height", d => d.y1 - d.y0)
                .attr("fill", "transparent")
                .attr("stroke", 0);
            
            let mouseover = function(d) {
                if (d.data.children.length > 0) {
                    d3.select(this)
                        .attr("fill", "white")
                        .attr("opacity", "0.2")
                        .attr("cursor", "pointer")
                }
            }
            
            let mouseout = function(d) {
                d3.select(this)
                    .attr("fill", "transparent")
                    .attr("opacity", null)
                    .attr("cursor", null)
            }
            
            hover_rects.on("mouseover", mouseover);
            hover_rects.on("mouseout", mouseout);
            
            // Now we create and bind the click event.
            let click = function(d) {
                if (d.data.children.length > 0) {
                    // Use the hover ID to get the underlying name attr (e.g. "Noise-hover" -> "Noise")
                    let name = this.getAttribute("id").slice(0,-6);
                    
                    // Build a new layout.
                    // TODO...
                    
                }
            }
            
            hover_rects.on("click", click);
                        
        });
        
    });
</script>

In [69]:
%%d3 4.4.4

<svg width=800 height=400></svg>

<script>
    requirejs.config({
        paths: {
            tree: "https://unpkg.com/threshold-tree@1.0.4/umd/threshold-tree"
        }
    });
    
    requirejs(['tree'], function(tree) {
        
        d3.csv("../data/complaint_types.csv", function(complaints) {
            
            ///////////////////////
            // INITIAL DATA LOAD //
            ///////////////////////
            
            // Build the threshold tree and convert it to a simple JSON format.
            let tr = new tree.ThresholdTree(complaints);
            
            // tr.root = tr.root.children[0];  // sub-example
            // tr.root = tr.root.children[0].children[6];  // sub-sub-example
            
            // Convert that to a hierarchy.
            let hr = tr.as_hierarchy();
            
            let root = d3.stratify().id(d => d.name).parentId(d => d.parent)(hr);
            
            // Attach a summer to the hierarchy.
            // This summer needs to account for two factors:
            // 1. It should not include the n of the root node, as this will double to display space.
            // 2. It should not include sub-sub-children, as D3 recurses naively and doesn't care about
            //    how deep our objects are.
            // These two things were discovered experimentally.
            // Unfortunately, Set.has doesn't work on Objects. See:
            // http://stackoverflow.com/questions/29759480/how-to-customize-object-equality-for-javascript-set
            // So we have to implement a method for checking if an Object is a subchild ourselves. This is
            // the below.
            //
            // Note: a simpler solution is to simply replace the list of children with an empty list back when
            // we first made the list. e.g.: tr.root.children.forEach(function(node) { node.children = []; });,
            // but we need to keep that data around for transitions, not just delete it, so we don't do that.
            function object_is_direct_child(root, obj) {
                return (root.children.filter(child => (child.id === obj.name)).length > 0);
            }
            
            // Now the sum function itself.
            function nodal_summer(node) {
                return ((object_is_direct_child(root, node)) ? node.n : 0);
            }
            
            root = root.sum(nodal_summer);
            
            // Create our treemap layout factory function.
            let treemap = d3.treemap()
                .tile(d3.treemapResquarify)
                .size([800, 400])
                .round(true)
                .paddingInner(1);
            
            // Apply our treemap function to our data.
            treemap(root);

            
            ////////////////////////////
            // POPULATING THE DISPLAY //
            ////////////////////////////            
            
            // Select the SVG.
            const svg = d3.select("svg");
            
            // To lay out the cells, we would typically make use of root.leaves(), which fetches all
            // of the leaf nodes of the hierarchy and returns them. This is used for the data bind.
            // However, in our case we don't want the leaves, because these are the items at the bottom
            // of our hierarchy. We use root.children directly instead. Works fine.
            
            // Lay out the cells (element transform on groups)
            let cells = svg.selectAll("g")
                .data(root.children)
                .enter()
                .append("g")
                .attr("transform", d => `translate(${d.x0}, ${d.y0})`);
            
            // Append rects.
            let rects = cells.append("rect")
                .attr("id", d => `${d.data.name}-rect`)
                .attr("width", d => d.x1 - d.x0)
                .attr("height", d => d.y1 - d.y0)
                .attr("fill", "steelblue")
                .attr("stroke", "white")
                .attr("stroke-width", "1");
            
            // Append text labels.
            let texts = cells.append("text")
                .attr("id", d => `${d.data.name}-text`)
                .attr("transform", d => `translate(${(d.x1 - d.x0) / 2}, ${(d.y1 - d.y0) / 2})`)
                .attr("text-anchor", "middle")
                .attr("alignment-baseline", "middle")
                .text(d => d.data.name)
            
            
            //////////////////////
            // MOUSEOVER EVENTS //
            //////////////////////
            
            // Add mouseover events signaling interactivity. We want these to fire regardless of whether
            // we are hoving over the text or the rect behind it, so we add invisible rects (same as the)
            // visible ones) to the screen on top, which have hover effects defined on them.
            let hover_rects = cells.append("rect")
                .attr("id", d => `${d.data.name}-hover`)
                .attr("width", d => d.x1 - d.x0)
                .attr("height", d => d.y1 - d.y0)
                .attr("fill", "transparent")
                .attr("stroke", 0);
            
            let mouseover = function(d) {
                if (d.data.children.length > 0) {
                    d3.select(this)
                        .attr("fill", "white")
                        .attr("opacity", "0.2")
                        .attr("cursor", "pointer")
                }
            }
            
            let mouseout = function(d) {
                d3.select(this)
                    .attr("fill", "transparent")
                    .attr("opacity", null)
                    .attr("cursor", null)
            }
            
            hover_rects.on("mouseover", mouseover);
            hover_rects.on("mouseout", mouseout);
            
            //////////////////
            // CLICK EVENTS //
            //////////////////
            
            let click = function(d) {
                if (d.data.children.length > 0) {
                    //
                    // First we build the layout of the cells that we will want at the end of the transition.
                    //
                    
                    // Use the hover ID to get the underlying name attr (e.g. "Noise-hover" -> "Noise")
                    let name = this.getAttribute("id").slice(0,-6);
                    
                    // Select the new root node.
                    let new_root = root.children.find(c => c.id === name);
                    
                    // Adjust the root depth values---these are used in layout.
                    new_root.eachBefore(function(node) { node.depth--; })

                    // Sum the values...
                    new_root = treemap(new_root.sum(node => ((object_is_direct_child(new_root, node)) ? node.n : 0)));
                    
                    // ...and lay out the map.
                    treemap(new_root);
                    
                    cells.transition()
                        .duration(750)
                        .attr("transform", function(d) { 
                            console.log(d.x0, d.y0);
                            return "translate(" + d.x0 + "," + d.y0 + ")"; 
                        })                    
                    
                    //
                    // Next we build the layout of the cells that we want during the transition.
                    //
                    
                }
            }
            
            hover_rects.on("click", click);
            
        });
        
    });
</script>

In [116]:
%%d3 4.4.4

<svg width=800 height=400></svg>

<script>
    requirejs.config({
        paths: {
            tree: "https://unpkg.com/threshold-tree@1.0.4/umd/threshold-tree"
        }
    });
    
    requirejs(['tree'], function(tree) {
        
        d3.csv("../data/complaint_types.csv", function(complaints) {
            
            ///////////////////////
            // INITIAL DATA LOAD //
            ///////////////////////
            
            // Build the threshold tree and convert it to a simple JSON format.
            let tr = new tree.ThresholdTree(complaints);
            
            // tr.root = tr.root.children[0];  // sub-example
            // tr.root = tr.root.children[0].children[6];  // sub-sub-example
            
            // Convert that to a hierarchy.
            let hr = tr.as_hierarchy();
            let root = d3.stratify().id(d => d.name).parentId(d => d.parent)(hr);
            
            //////////////////////
            // LAYOUT FUNCTIONS //
            //////////////////////
            
            // A filter function for the summer, accounting for two things:
            // 1. It should not include the n of the root node, as this will double to display space.
            // 2. It should not include sub-sub-children, as D3 recurses naively and doesn't care about
            //    how deep our objects are.
            function object_is_direct_child(root, obj) {
                // return (root.children.filter(child => (child.id === obj.name)).length > 0);
                return (root.children.filter(child => (child.data.name === obj.name)).length > 0);
            }
            
            // Calculates a layout for a root.
            function compute_layout(root, width=800, height=400, x0=0, y0=0) {
                // Make a copy (to maintain idempotence).
                let new_root = root.copy();
                
                // Create the layout function.
                let treemap = d3.treemap()
                    .tile(d3.treemapResquarify)
                    .size([width, height])
                    .round(true)
                    .paddingInner(1);
                
                // Create the summer.
                function summer(node) {
                    return ((object_is_direct_child(new_root, node)) ? node.n : 0);
                }
                
                // Apply the summer and the treemap.
                new_root = treemap(new_root.sum(summer));
                
                // Adjust the starting point.
                new_root.eachBefore(function(node) { 
                    node.x0 += x0; node.x1 += x0;
                    node.y0 += y0; node.y1 += y0; 
                }) 
                
                return new_root;                
            }
            
            // A reusable function for drawing individual treemap layouts.
            function draw_treemap(root) {
                // Select the SVG.
                const svg = d3.select("svg");

                // Lay out the cells (element transform on groups)
                let cells = svg.selectAll("g")
                    .data(root.children)
                    .enter()
                    .append("g")
                    .attr("transform", d => `translate(${d.x0}, ${d.y0})`);
                
                // Append rects.
                let rects = cells.append("rect")
                    .attr("id", d => `${d.data.name}-rect`)
                    .attr("width", d => d.x1 - d.x0)
                    .attr("height", d => d.y1 - d.y0)
                    .attr("fill", "steelblue")
                    .attr("stroke", "white")
                    .attr("stroke-width", "1");
            }
            
            ////////////////////////////
            // POPULATING THE DISPLAY //
            ////////////////////////////
            
            // Select the SVG.
            const svg = d3.select("svg");
            
            // Compute the initial layout.
            root = compute_layout(root);
            
            root.children.forEach(function(node) {
                let layout = compute_layout(node, width=(node.x1 - node.x0), height=(node.y1 - node.y0), 
                                            x0=node.x0, y0=node.y0);
                
                // console.log(layout);
                
                let cells = svg.append("g")
                    .selectAll("g")
                    .data(layout.children)
                    .enter()
                    .append("rect")
                    .attr("transform", d => `translate(${d.x0}, ${d.y0})`)
                    .attr("id", d => `${d.data.name}-rect`)
                    .attr("width", d => d.x1 - d.x0)
                    .attr("height", function(d) { return d.y1 - d.y0; })//d => d.y1 - d.y0)
                    .attr("fill", "steelblue")
                    .attr("stroke", "white")
                    .attr("stroke-width", "1");
                
            });
            
//            // Append text labels.
//            let texts = cells.append("text")
//                .attr("id", d => `${d.data.name}-text`)
//                .attr("transform", d => `translate(${(d.x1 - d.x0) / 2}, ${(d.y1 - d.y0) / 2})`)
//                .attr("text-anchor", "middle")
//                .attr("alignment-baseline", "middle")
//                .text(d => d.data.name)
            
            
            //////////////////////
            // MOUSEOVER EVENTS //
            //////////////////////
            
            // Add mouseover events signaling interactivity. We want these to fire regardless of whether
            // we are hoving over the text or the rect behind it, so we add invisible rects (same as the)
            // visible ones) to the screen on top, which have hover effects defined on them.
            let hover_rects = cells.append("rect")
                .attr("id", d => `${d.data.name}-hover`)
                .attr("width", d => d.x1 - d.x0)
                .attr("height", d => d.y1 - d.y0)
                .attr("fill", "transparent")
                .attr("stroke", 0);
            
            let mouseover = function(d) {
                if (d.data.children.length > 0) {
                    d3.select(this)
                        .attr("fill", "white")
                        .attr("opacity", "0.2")
                        .attr("cursor", "pointer")
                }
            }
            
            let mouseout = function(d) {
                d3.select(this)
                    .attr("fill", "transparent")
                    .attr("opacity", null)
                    .attr("cursor", null)
            }
            
            hover_rects.on("mouseover", mouseover);
            hover_rects.on("mouseout", mouseout);
            
            //////////////////
            // CLICK EVENTS //
            //////////////////
            
            let click = function(d) {
                if (d.data.children.length > 0) {
                    //
                    // First we build the layout of the cells that we will want at the end of the transition.
                    //
                    
                    // Use the hover ID to get the underlying name attr (e.g. "Noise-hover" -> "Noise")
                    let name = this.getAttribute("id").slice(0,-6);
                    
                    // Select the new root node.
                    let new_root = root.children.find(c => c.id === name);
                    
                    // Adjust the root depth values---these are used in layout.
                    new_root.eachBefore(function(node) { node.depth--; })

                    // Sum the values...
                    new_root = treemap(new_root.sum(node => ((object_is_direct_child(new_root, node)) ? node.n : 0)));
                    
                    // ...and lay out the map.
                    treemap(new_root);
                    
                    // cells.transition()
                    //     .duration(750)
                    //     .attr("transform", function(d) { 
                    //         console.log(d.x0, d.y0);
                    //         return "translate(" + d.x0 + "," + d.y0 + ")"; 
                    //     })                    
                    
                    //
                    // Next we build the layout of the cells that we want during the transition.
                    //
                    
                }
            }
            
            hover_rects.on("click", click);
            
        });
        
    });
</script>